In [1]:
# Datasets 
#     - Cluster allocation
#     - transforms
 

# Model loaded 


In [8]:
%load_ext autoreload
%autoreload 2

import os 
import sys 
sys.path.append("../")
import argparse
from glob import glob
import math
import numpy as np
import torch
import torch.nn as nn
import torchvision
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# df_submission = pd.read_csv(f"../starting_k/pred_{dataset_name}.csv")
# df_submission

In [4]:
"../starting_k/pred_aircraft.csv"

'../starting_k/pred_aircraft.csv'

In [5]:
def get_img_paths(dataset_name):
    """
    Arguments:
    dataset_name = {"scars", "fgvc_aircraft", "cub"}
    """
    if dataset_name == "fgvc_aricraft":
        df_submission = pd.read_csv(f"../starting_k/pred_aircraft.csv")
    else: 
        df_submission = pd.read_csv(f"../starting_k/pred_{dataset_name}.csv")
    
    glob_format = {
            "scars" : "../dataset/Stanford_Cars/car_data/car_data/*/*/*.jpg",
            "fgvc_aricraft" : "../dataset/FGVC_Aircraft/fgvc-aircraft-2013b/data/images/*.jpg",
            "cub" : "../dataset/CUB/CUB_200_2011/images/*/*.jpg"
        }
    whole_data_list = glob(glob_format[dataset_name])
    name_extract = [x.split("/")[-1] for x in whole_data_list]
    
    img_paths = []
    for sample in df_submission["img"]:

        idx = name_extract.index(sample)

        if idx is not None:
            img_paths.append(whole_data_list[idx])

        else:
            print(sample)
            return None

    return img_paths


from PIL import Image


def pil_loader(path: str) -> Image.Image:
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
class SubmissionImageFolder():

    """
    Base ImageFolder
    """

    def __init__(self, paths, transform):

        self.paths = paths
        self.transform = transform
        self.target_transform = None      

    def __len__(self):
        return len(self.paths)
    
    
    def __getitem__(self, item):

        img = pil_loader(self.paths[item])
        if self.transform is not None:
            img = self.transform(img)
        return img


In [6]:
%load_ext autoreload
%autoreload 2

import os 
import sys 
sys.path.append("../")
import argparse

import math
import numpy as np
import torch
import torch.nn as nn
from torch.optim import SGD, lr_scheduler
from torch.utils.data import DataLoader
from tqdm import tqdm

from data.augmentations import get_transform
from data.get_datasets import get_datasets, get_class_splits

import SSB
from SSB.get_datasets.get_gcd_datasets_funcs import get_gcd_datasets
from SSB.utils import load_class_splits

from util.general_utils import AverageMeter, init_experiment
from util.cluster_and_log_utils import log_accs_from_preds
from config import exp_root
from model import DINOHead, info_nce_logits, SupConLoss, DistillLoss, ContrastiveLearningViewGenerator, get_params_groups
from data.cub import get_cub_datasets



parser = argparse.ArgumentParser(description='cluster', formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--num_workers', default=8, type=int)
parser.add_argument('--eval_funcs', nargs='+', help='Which eval functions to use', default=['v2', 'v2p'])

parser.add_argument('--warmup_model_dir', type=str, default=None)
parser.add_argument('--dataset_name', type=str, default='scars', help='options: cifar10, cifar100, imagenet_100, cub, scars, fgvc_aricraft, herbarium_19')
parser.add_argument('--prop_train_labels', type=float, default=0.5)
parser.add_argument('--use_ssb_splits', action='store_true', default=True)

parser.add_argument('--grad_from_block', type=int, default=11)
parser.add_argument('--lr', type=float, default=0.1)
parser.add_argument('--gamma', type=float, default=0.1)
parser.add_argument('--momentum', type=float, default=0.9)
parser.add_argument('--weight_decay', type=float, default=1e-4)
parser.add_argument('--epochs', default=200, type=int)
parser.add_argument('--exp_root', type=str, default=exp_root)
parser.add_argument('--transform', type=str, default='imagenet')
parser.add_argument('--sup_weight', type=float, default=0.35)
parser.add_argument('--n_views', default=2, type=int)

parser.add_argument('--memax_weight', type=float, default=2)
parser.add_argument('--warmup_teacher_temp', default=0.07, type=float, help='Initial value for the teacher temperature.')
parser.add_argument('--teacher_temp', default=0.04, type=float, help='Final value (after linear warmup)of the teacher temperature.')
parser.add_argument('--warmup_teacher_temp_epochs', default=30, type=int, help='Number of warmup epochs for the teacher temperature.')

parser.add_argument('--fp16', action='store_true', default=False)
parser.add_argument('--print_freq', default=10, type=int)
parser.add_argument('--exp_name', default=None, type=str)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/conor/SimGCD-ICCV-Challenge/SSB/SSB/utils.py:55: UserWarning: The dataset path is set to the default value. Consider setting it to a suitable path in your ~/.ssb/ssb_config.json file.
  warnings.warn("The dataset path is set to the default value. "


_StoreAction(option_strings=['--exp_name'], dest='exp_name', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [10]:
def predict(model, test_loader):

    model.eval()

    preds = []

    for batch_idx, images in enumerate(tqdm(test_loader)):
        images = images.cuda(non_blocking=True)
        
        with torch.no_grad():
            _, logits = model(images)
            preds.append(logits.argmax(1).cpu().numpy())

    preds = np.concatenate(preds)

    return preds

In [12]:
args = parser.parse_args(args=[])


dataset_names = ["scars", "cub", "fgvc_aricraft"]
args.dataset_name = dataset_names[1]
print(args.dataset_name)
# ----------------------
# INIT
# ----------------------

device = torch.device('cuda:0')
if 'imagenet' in args.dataset_name:
    args.mlp_out_dim = 1000

torch.backends.cudnn.benchmark = True

# ----------------------
# BASE MODEL
# ----------------------
args.interpolation = 3
args.crop_pct = 0.875

backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')

args.image_size = 224
args.feat_dim = 768
args.num_mlp_layers = 3

class_splits = load_class_splits(args.dataset_name)

args.train_classes = class_splits['known_classes']
args.mlp_out_dim = len(class_splits['known_classes']) \
                 + len(class_splits['unknown_classes']['Easy']) \
                 + len(class_splits['unknown_classes']['Medium']) \
                 + len(class_splits['unknown_classes']['Hard'])

# ----------------------
for m in backbone.parameters():
    m.requires_grad = False


train_transform, test_transform = get_transform(args.transform, image_size=args.image_size, args=args)
# --------------------
# DATASETS
# --------------------
img_paths = get_img_paths(args.dataset_name)
submission_dataset = SubmissionImageFolder(img_paths, train_transform)


projector = DINOHead(in_dim=args.feat_dim, out_dim=args.mlp_out_dim, nlayers=args.num_mlp_layers)
model = nn.Sequential(backbone, projector).to(device)

model_paths = {
    "cub" :  '../dev_outputs/simgcd/log/cub_simgcd_(21.08.2023_|_38.777)/checkpoints/model.pt',
    "scars" : '../dev_outputs/simgcd/log/scars_simgcd_(21.08.2023_|_38.896)/checkpoints/model.pt',
    "fgvc_aricraft" : '../dev_outputs/simgcd/log/aircraft_simgcd_(21.08.2023_|_38.860)/checkpoints/model.pt'
}


model_path = model_paths[args.dataset_name]

print("LOAD MODEL : ", model_path )
model.load_state_dict(torch.load(model_path, map_location=device)["model"])
# if args.warmup_model_dir is not None:
#     backbone.load_state_dict(torch.load(args.warmup_model_dir, map_location='cpu'))


submission_loader = DataLoader(submission_dataset, 
                         num_workers=args.num_workers, 
                         batch_size=256, 
                         shuffle=False, 
                         pin_memory=False)


predictions = predict(model, submission_loader)

if args.dataset_name == "fgvc_aricraft":
    df_submission = pd.read_csv(f"../starting_k/pred_aircraft.csv")
else: 
    df_submission = pd.read_csv(f"../starting_k/pred_{args.dataset_name}.csv")

df_submission['pred'] = predictions
df_submission.to_csv(f"./sample_{args.dataset_name}.csv", index=False)
df_submission

cub


Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


LOAD MODEL :  ../dev_outputs/simgcd/log/cub_simgcd_(21.08.2023_|_38.777)/checkpoints/model.pt


100%|██████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:13<00:00,  1.37it/s]


,img,pred
0,Black_Footed_Albatross_0009_34.jpg,46
1,Black_Footed_Albatross_0014_89.jpg,183
2,Black_Footed_Albatross_0031_100.jpg,188
3,Black_Footed_Albatross_0040_796066.jpg,111
4,Black_Footed_Albatross_0089_796069.jpg,111
...,...,...
4491,Common_Yellowthroat_0080_190663.jpg,4
4492,Common_Yellowthroat_0087_190414.jpg,4
4493,Common_Yellowthroat_0098_190430.jpg,4
4494,Common_Yellowthroat_0094_190690.jpg,4
